In [2]:
import pandas as pd
import numpy as np

In [181]:
data_matches = pd.read_csv("files/WorldCupMatches.csv")
data_players = pd.read_csv("files/WorldCupPlayers.csv")
data_wc = pd.read_csv("files/WorldCups.csv")

## Matches DB

In [182]:
# check data
data_matches.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,4444.0,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,18346.0,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,24059.0,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,,2549.0,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,,23409.0,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA


In [186]:
# some of the 2014 matches have multiple entries which we must remove
data_matches[data_matches["MatchID"].isin(data_matches["MatchID"][data_matches["MatchID"].duplicated()])]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials


In [184]:
# Remove AR entry for 2014 final since it will not be removed by the func
data_matches.drop(851, inplace=True)

In [185]:
# Remove Repeated entries
data_matches.drop_duplicates(inplace=True)

In [187]:
# Drop some columns by eye
data_matches.drop(['Referee', 'Assistant 1', 'Assistant 2', 'RoundID', 'MatchID', 'Attendance', 'Stadium', 'City', 'Datetime'], axis=1, inplace=True)

In [188]:
# Functions that will help cleaning the dataset
def get_winner(df):
    """
    Get the winner from the dataset based on goals
    """
    if df['Home Team Goals'] > df['Away Team Goals']:
        return df['Home Team Initials']
    return df['Away Team Initials']

def get_loser(df):
    """
    Get the loser from the dataset based on goals
    """
    if df['Home Team Goals'] < df['Away Team Goals']:
        return df['Home Team Initials']
    return df['Away Team Initials']


def clean_stage(df):
    """
    Remove the groups x and replace it just with Groups
    """
    if df['Stage'].__contains__('Group'):
        return 'Group'
    return df['Stage']

def loser(rows):
    c = 0
    for row in rows:
        print(row)
        if row["Loser Team"] == row['Winner Team']:
            c += 1

    return c 

In [189]:
data_matches['Winner Team'] = data_matches.apply(lambda row: get_winner(row), axis=1)
data_matches['Loser Team'] = data_matches.apply(lambda row: get_loser(row), axis=1)
data_matches['Stage'] = data_matches.apply(lambda row: clean_stage(row), axis=1)

In [221]:
loses_count = data_matches.groupby(["Loser Team", "Stage"], as_index=False)["Year"].count().rename(columns={'Year': 'Loses', "Loser Team": "Country"})

In [222]:
winner_count = data_matches.groupby(["Winner Team", "Stage"], as_index=False)["Year"].count().rename(columns={"Winner Team": "Country", 'Year': 'Wins'})

In [225]:
agg_data = data_matches.groupby(["Winner Team", "Stage"], as_index=False)['Home Team Goals', "Away Team Goals", "Half-time Home Goals", "Half-time Away Goals"].mean().rename(columns={"Winner Team": "Country"})

/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_87890/1476354521.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_data = data_matches.groupby(["Winner Team", "Stage"], as_index=False)['Home Team Goals', "Away Team Goals", "Half-time Home Goals", "Half-time Away Goals"].mean().rename(columns={"Winner Team": "Country"})


In [226]:
import functools as ft
data_matches_final = ft.reduce(lambda left, right: pd.merge(left, right, on=['Country', "Stage"]), [loses_count, winner_count, agg_data])
data_matches_final

,Country,Stage,Loses,Wins,Home Team Goals,Away Team Goals,Half-time Home Goals,Half-time Away Goals
0,ALG,Group,7,4,1.500000,2.000000,0.750000,0.750000
1,ANG,Group,3,2,0.500000,0.500000,0.000000,0.000000
2,ARG,Final,3,2,3.000000,1.500000,0.500000,0.000000
3,ARG,Group,18,36,2.250000,0.611111,1.083333,0.250000
4,ARG,Quarter-finals,5,4,1.000000,0.500000,0.250000,0.000000
...,...,...,...,...,...,...,...,...
132,USA,Round of 16,3,1,0.000000,2.000000,0.000000,1.000000
133,WAL,Group,3,4,1.000000,0.750000,0.250000,0.750000
134,YUG,Group,12,19,2.473684,0.789474,1.000000,0.368421
135,YUG,Quarter-finals,2,1,1.000000,0.000000,0.000000,0.000000
